## Análisis de Resultados y Mejora con Sesgos

| Modelo          | MAE    | RMSE   | Precision | Recall  | F1      | nDCG@10 |
|-----------------|--------|--------|-----------|---------|---------|---------|
| Baseline (User-Mean) | 0.99   | 0.77   | 0.92      | 1.00    | 0.96    | 0.12    |
| PMF “puro”      | 0.77   | 0.99   | 0.53      | 0.54    | 0.53    | 0.05    |
| PMF + Sesgos    | 0.68   | 0.86   | 0.93      | 0.98    | 0.96    | 0.96    |
| BeMF (Bernoulli)| 1.56   | 1.95   | 0.69      | 0.40    | 0.51    | 0.2    |

1. **Mejora de MAE/RMSE**:  
   - Incluir sesgos de usuario e ítem corrige gran parte del error sistemático.  
   - Pasamos de MAE≈0.77/RMSE≈0.99 a MAE≈0.68/RMSE≈0.86, lo cual es una mejora sustancial.

2. **Clasificación (Precision/Recall/F1)**:  
   - El umbral dinámico (floor(media global)) ya funcionaba bien en el baseline, pero el PMF puro sólo alcanza F1≈0.53.  
   - Con sesgos, F1 sube a ≈0.96, indicando que el modelo “sesgado” ajusta casi perfectamente quién dará like/dislike.
   - Hemos de tener en cuenta que este umbral es, quizá, demasiado bajo, (3), pero al subirlo al 4 el recall era demasiado bajo, por lo que fuimos
   algo más laxos en lo que significa el like.

3. **Ranking (nDCG@10)**:  
   - El baseline alcanza ≈0.12, PMF puro ≈0.05 (ranking casi aleatorio) y PMF+Bias ≈0.96 (ranking casi perfecto).  
   - BeMF no converge en 10 iteraciones pues optimiza probabilidad, no orden.

---

### Advertencias y puntos de atención

- **Elección del umbral**: un threshold bajo para la binarización afecta las métricas como F1 o recall.  
- **Partición train/test**: debimos asegurarnos de que no hubiera filtrado de usuarios del train al test.  
- **Sobreajuste**: evitar que el PMF con sesgo evite memorizar casos para mejorar nDCG.  
- **Iteraciones y tasa de aprendizaje**: BeMF y PMF necesitan más tamaño de paso o iteraciones para converger.  


## Comparativa de Modelos

- **KNN (sample de 1 000 usuarios/libros):** con pocos datos se mantiene alta precisión (0.83) pero muy bajo recall (0.24).  
- **PMF “puro”:** converge rápido en MAE/RMSE, pero su ordenamiento en nDCG (0.12) necesita más iteraciones para afinarse.  
- **BeMF (Bernoulli MF):** tras 10 iteraciones alcanza nDCG=0.20, pero requiere muchas más pasadas para converger bien en ranking.  
- **PMF + Bias:** corrige sesgos globales, de usuario e ítem, logrando el mejor equilibrio entre regresión y ranking con pocas iteraciones (nDCG=0.96).  

**Conclusión:** para conjuntos pequeños y pocas iteraciones, PMF+Bias es la opción más robusta; Bernoulli necesita más ciclos para un ranking competitivo, y KNN exige muestras grandes para mejorar el recall.